In [1]:
!pip install gensim

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os.path
embedding_dim = 200
filepath = '/content/drive/MyDrive/KTO_Project/data/'

# 그래프에서 한글표현을 위해 폰트를 설치
%config InlineBackend.figure_format = 'retina'       # 폰트가 깨끗하게 보이도록 설정
!apt install fonts-nanum                             # 나눔 폰트 설치
#!apt-get -qq -y install fonts-nanum > /dev/null     # 폰트 설치 대안 코드

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

# 기본 글꼴 변경
mpl.pyplot.rc('font', family='NanumMyeongjo')
mpl.font_manager._rebuild()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (7,911 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [3]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1
!pip install rhinoMorph

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,426 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http:

## 리뷰 불러오기

In [4]:
df = pd.read_excel('/content/drive/MyDrive/KTO_Project/data/hanspell_review.xlsx')

In [5]:
df

,name,review,rate,hanspell_review
0,월인당[한국관광 품질인증/Korea Quality],"지난 주말, 부모님과 함께 널찍한 노마루가 있는 산노을 방에서 묵었어요. 꽃피는 봄...",5.00,"지난 주말, 부모님과 함께 널찍한 노마루가 있는 산노을 방에서 묵었어요. 꽃피는 봄..."
1,와담정[한국관광 품질인증/Korea Quality],월정교와도 가깝고 방도 넘 깨끗했습니다. 사장님도 너무 친절하시고 좋았어요. 특히 ...,5.00,월정교와도 가깝고 방도 넘 깨끗했습니다. 사장님도 너무 친절하시고 좋았어요. 특히 ...
2,와담정[한국관광 품질인증/Korea Quality],목조 한옥이다보니 벌레가 너무 많았어요. 톡톡이라고 깨알보다 작은 벌레들이 이불에도...,2.65,목조 한옥이다보니 벌레가 너무 많았어요. 톡톡이라고 깨알보다 작은 벌레들이 이불에도...
3,비센트호텔[한국관광 품질인증/Korea Quality],"아주 늦은시간 도착했는데 프론트 친절하시고, 객실정비도 잘 되어있고 수건도 넉넉해...",5.00,"아주 늦은시간 도착했는데 프론트 친절하시고, 객실정비도 잘 되어있고 수건도 넉넉해..."
4,비센트호텔[한국관광 품질인증/Korea Quality],가격대비 만족합니다호텔도 깨끗한 편이에요,5.00,가격대비 만족합니다호텔도 깨끗한 편이에요
...,...,...,...,...
115139,공주한옥마을[한국관광 품질인증/Korea Quality],팀 워크숍을 준비하면서 어디가 적당할까하고 조사해본 결과 공주한옥마을을 선정했다. ...,5.00,팀 워크숍을 준비하면서 어디가 적당할까하고 조사해본 결과 공주한옥마을을 선정했다. ...
115140,공주한옥마을[한국관광 품질인증/Korea Quality],어머니와 함께 사곡관 (공동 욕실과 화장실을 쓰는 개별방) 에 1박 투숙하였습니다....,4.00,어머니와 함께 사곡관 (공동 욕실과 화장실을 쓰는 개별방) 에 1박 투숙하였습니다....
115141,공주한옥마을[한국관광 품질인증/Korea Quality],지난 5월 가족여행으로 1박2일 공주를 다녀왔습니다. 공주 '한옥마을 정안관'에 숙...,5.00,지난 5월 가족여행으로 1박2일 공주를 다녀왔습니다. 공주 '한옥마을 정안관'에 숙...
115142,공주한옥마을[한국관광 품질인증/Korea Quality],지은지 얼마 안된 건물처럼 깨끗한 한옥시설을 갖추었다. 화장실도 신식으로 되어 있고...,5.00,지은지 얼마 안된 건물처럼 깨끗한 한옥시설을 갖추었다. 화장실도 신식으로 되어 있고...


In [6]:
df[df['hanspell_review'].isna()]
df = df.replace('\n', ' ')

In [7]:
#4. RHINO 시작
import rhinoMorph
rn = rhinoMorph.startRhino()

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!


In [8]:
md_df = df['hanspell_review'] 
print(md_df)

0         지난 주말, 부모님과 함께 널찍한 노마루가 있는 산노을 방에서 묵었어요. 꽃피는 봄...
1         월정교와도 가깝고 방도 넘 깨끗했습니다. 사장님도 너무 친절하시고 좋았어요. 특히 ...
2         목조 한옥이다보니 벌레가 너무 많았어요. 톡톡이라고 깨알보다 작은 벌레들이 이불에도...
3         아주 늦은시간 도착했는데  프론트 친절하시고, 객실정비도 잘 되어있고 수건도 넉넉해...
4                                    가격대비 만족합니다호텔도 깨끗한 편이에요
                                ...                        
115139    팀 워크숍을 준비하면서 어디가 적당할까하고 조사해본 결과 공주한옥마을을 선정했다. ...
115140    어머니와 함께 사곡관 (공동 욕실과 화장실을 쓰는 개별방) 에 1박 투숙하였습니다....
115141    지난 5월 가족여행으로 1박2일 공주를 다녀왔습니다. 공주 '한옥마을 정안관'에 숙...
115142    지은지 얼마 안된 건물처럼 깨끗한 한옥시설을 갖추었다. 화장실도 신식으로 되어 있고...
115143    Didnt는 정말로 머무른다, 단지 모양을보기 위해 갔다 - 보라. 별로 아무것도,...
Name: hanspell_review, Length: 115144, dtype: object


## 텍스트 형태소 분석

In [9]:
#Word2Vec – 텍스트의 형태소 분석
total_lines = len(md_df)
md_rev = []
cnt=0
with open(filepath+'morphed.txt', 'w', encoding='utf-8')as f:
  for data_each in md_df:
    morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
    joined_data_each = ' '.join(morphed_data_each)
    if joined_data_each:
      f.write(joined_data_each + '\n')
    cnt += 1
    if (cnt % 1000) == 0:  # 진행 정도 확인을 위해 1000번째 문장마다 확인
      print(round(cnt/total_lines * 100, 3), '%')
  print('Morphological Analysis Completed.')

0.868 %
1.737 %
2.605 %
3.474 %
4.342 %
5.211 %
6.079 %
6.948 %
7.816 %
8.685 %
9.553 %
10.422 %
11.29 %
12.159 %
13.027 %
13.896 %
14.764 %
15.633 %
16.501 %
17.37 %
18.238 %
19.107 %
19.975 %
20.843 %
21.712 %
22.58 %
23.449 %
24.317 %
25.186 %
26.054 %
26.923 %
27.791 %
28.66 %
29.528 %
30.397 %
31.265 %
32.134 %
33.002 %
33.871 %
34.739 %
35.608 %
36.476 %
37.345 %
38.213 %
39.081 %
39.95 %
40.818 %
41.687 %
42.555 %
43.424 %
44.292 %
45.161 %
46.029 %
46.898 %
47.766 %
48.635 %
49.503 %
50.372 %
51.24 %
52.109 %
52.977 %
53.846 %
54.714 %
55.583 %
56.451 %
57.32 %
58.188 %
59.056 %
59.925 %
60.793 %
61.662 %
62.53 %
63.399 %
64.267 %
65.136 %
66.004 %
66.873 %
67.741 %
68.61 %
69.478 %
70.347 %
71.215 %
72.084 %
72.952 %
73.821 %
74.689 %
75.558 %
76.426 %
77.295 %
78.163 %
79.031 %
79.9 %
80.768 %
81.637 %
82.505 %
83.374 %
84.242 %
85.111 %
85.979 %
86.848 %
87.716 %
88.585 %
89.453 %
90.322 %
91.19 %
92.059 %
92.927 %
93.796 %
94.664 %
95.533 %
96.401 %
97.27 %
98.138 %
99.006 

## 형태소 분석 결과 읽어 리스트로 만들기 - Word2Vec

In [10]:
def read_data(filename, encoding='utf-8'):
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split(' ') for line in f.read().splitlines()]
  return data

data = read_data(filepath+'morphed.txt', 'utf-8')

print(len(data))
print(type(data))
print(data[:3])

114259
<class 'list'>
[['지나', '주말', '부모', '함께', '널찍', '노마', '루가', '산', '노을', '방', '묵', '꽃피', '봄', '오', '떨어지', '좀', '쌀쌀', '날씨', '뜨끈뜨끈', '하', '구들장', '아랫목', '덕분', '정말', '정말', '따뜻', '자', '여행', '피로도', '풀', '시골집', '오', '정겨움', '느끼', '숙소', '밤하늘', '별도', '완전', '잘', '보이', '느긋', '자연', '속', '한옥', '체험', '원하', '강추', '하'], ['월정', '교', '가깝', '방도', '너무', '깨끗', '사장', '너무', '친절', '좋', '특히', '뚜벅', '체크아웃', '하', '때', '걸어가', '생각', '걱정', '사장', '태우', '감사', '다만', '한옥', '방음', '완벽', '되', '조용히', '얘기', '하', '옆방', '소리', '들리', 'ㅎㅎ', '벌레', '없', '에어컨', '보일러', '모두', '잘', '작동', '되', '덕분', '쾌적', '잘', '수', '있', '조식', '알', '만들', '빵', '과일', '우유', '주스', '컵라면', '계란', '많', '하루', '동안', '편안', '잘', '쉬'], ['목조', '한옥', '벌레', '너무', '많', '톡톡이', '깨알', '작', '벌레', '이불', '기어', '다니', '친구', '다', '불편', '하', '밖', '건물', '보', '때', '예쁘', '멋지', '방', '안', '오래되', '평범', '여름', '벌레', '힘들', 'ㅠ', '조식', '셀프', '해먹', '간단', '하', '음식', '체크아웃', '때', '차', '데리', '좋', '^^', '냉장고', '물', '있', '수건', '넉넉히', '있', '샴푸', '린스', '치약', '기본적', '인', '다', '있']]


## 임베딩 구성 - Word2Vec

In [11]:
from gensim.models import Word2Vec
os.chdir(filepath)

In [12]:
model = Word2Vec(sentences=data, size=embedding_dim, window=15, min_count=5, workers=4, sg=1)

In [13]:
model.save('embedding_skipgram.model')

## 임베딩 값 저장 - Word2Vec

In [14]:
words = list(model.wv.vocab)
with open('embedding_skipgram.txt', 'w') as f:
  for word in words:
    data = model.wv[word].tolist()
    print('data_pre: ', data)

    data.insert(0, word)
    print('data_after: ', data)

    for item in data:
      f.write('%s ' %item)
    f.write('\n')

Output hidden; open in https://colab.research.google.com to view.

## 유사어 찾기

In [15]:
model = Word2Vec.load('embedding_skipgram.model')

In [16]:
similarWords = model.wv.most_similar(positive=['위생'], topn=100)
print(similarWords)

[('청결', 0.6296083927154541), ('상태', 0.5897526741027832), ('겉보기', 0.5581031441688538), ('믿음직', 0.5574996471405029), ('속상', 0.5533879995346069), ('미비', 0.5504564046859741), ('양호', 0.5500437021255493), ('적인', 0.5455769300460815), ('퍼펙트', 0.5445213317871094), ('청소', 0.5431723594665527), ('물때', 0.5354629755020142), ('우선적', 0.5351496934890747), ('에비', 0.533613920211792), ('감점', 0.5334618091583252), ('찜찜', 0.5331672430038452), ('청경', 0.5326474905014038), ('따봉', 0.5326462388038635), ('압도적', 0.5313917398452759), ('물관리', 0.5293309688568115), ('극찬', 0.5293272733688354), ('누렇', 0.5291130542755127), ('광장히', 0.5284200310707092), ('이불상', 0.5272214412689209), ('결도', 0.5242626667022705), ('물청소', 0.5241978764533997), ('엉망', 0.523115873336792), ('도나', 0.5225383639335632), ('재채기', 0.5216180682182312), ('작용', 0.5214118361473083), ('방청', 0.5210140943527222), ('투성', 0.5184666514396667), ('드럽', 0.5151519179344177), ('거인', 0.5142926573753357), ('불만족', 0.5132958889007568), ('꾸준', 0.5119743943214417), ('우선시', 0.

In [17]:
word=[]
for similarWord in similarWords:
  word.append(similarWord[0])
print(word)

['청결', '상태', '겉보기', '믿음직', '속상', '미비', '양호', '적인', '퍼펙트', '청소', '물때', '우선적', '에비', '감점', '찜찜', '청경', '따봉', '압도적', '물관리', '극찬', '누렇', '광장히', '이불상', '결도', '물청소', '엉망', '도나', '재채기', '작용', '방청', '투성', '드럽', '거인', '불만족', '꾸준', '우선시', '이정', '핏자국', '소량', '팡', '인테리', '쓰이', '잘구', '이제껏', '읍', '면적', '최상인', '퐁퐁', '물질', '따로따로', '자재', '않', '솔직', '지저분', '밟히', '대도', '고침', '시설물', '성도', '방상태', '비품', '선입견', '꽝', '강북구', '흡', '소독', '매일매일', '너저분', '사용감', '첨가', '가운에', '위생적', '카펫트', '시큼', '버금가', '어쨋든', '수없이', '오리털', '상도', '가닥', '부스러기', '물컵', '중요시', '타인', '깐깐', '타호', '불량', '다말', '올', '성격', '보송보송', '딱하', '극상', '그다지', '아아아아아', '벗겨지', '살피', '찝찝', '덩', '뭉치']


## 두 단어 사이의 유사도 계산 - Word2Vec

In [18]:
print('위생과 청결 유사도: ', model.wv.similarity('입장료', '나빠'))

위생과 청결 유사도:  0.4169715


In [19]:
print('서비스 편의: ', model.wv.similarity('서비스', '편의'))

서비스 편의:  0.4426789


In [20]:
print('가격 착하: ', model.wv.similarity('가격', '착하'))

가격 착하:  0.38140222


In [21]:
model.wv.similarity('대구','부산')

0.5041747